## Julia Macros : Learning through use cases

If you are reading this, you either didn't read the Julia Meta Programming docs ; or you are a masochist. The docs are correct. Nobody in their right mind should be trying to create macros. They are too difficult. I didn't believe the docs. I **use** macros often. What would life be like without @btime; @match; @test; @kwdef. Their authors must be of sound mind. Surely macros can't be all that bad.

I took the plunge. I wrote one. It was easy. The docs lied. I took the plunge again. I almost drowned in crypic errors. The docs are right. Macros are almost impossible. I must be a masochist. I thought if I studdied a few macros I would figure out the patterns and life would be good. I followed the patterns. I stuck random $'s signs and "esc" in various places in my macros. Sometimes it even worked. How many missteps would it take to figure out the patterns?

### Disclaimer: Reading this is not going to make macros easy

Reading this will help you understand why macros are hard. It may encourge you to try harder to find a function that will do a simalar job, or that copy and paste isn't as bad as you thought it was. 

### Background: Why I choose the masochistic route.

I am building a DSL for composition. This is a passion project. It's first task the rather challenging task of composing mulutimedia artworks comprising of music and images. I am hoping for a natural and expressive way for the composer to provide input to the composition process. A natural process will make for easy learning and easy iterative revisions. Nothing kills the creative flow more than hunting down coding errors and fighting the syntax to work around them. I know that if I use macros, I have less constraints on the syntax. Given the dire warnings in the docs, I wanted to know how viable it would front my API with macros, hiding most of my functions and resticting the amount of Julia syntax that the composer needs to understand. 

### Metagoal: Life lessons

As technologists, we pride ourselves in what we know. We also pride ourselves in our ability to get stuff done. When our deep knowledge propells us to get stuff done better and faster, these two sources of pride are aligned. However when we favor what we know for expediant results, we actively avoid what we don't know. This is a big deal! It hampers our personal growth. It causes conflict in teams. I could go on..., but instead I will leave you with my abstract metagoal: to find techniques to embrace unknowns; get stuff done ; and make meaningful progress. **Meaningful progress seldom comes from what you know**.

### A gentle start

Not all macros are hard. When I got lucky with my first one, it wasn't luck. If all you need is a macro that substitutes a variable or two into templated code, you should be ok.

In [42]:
"""
Enable multiply for a struct with a .data vector
"""
macro vectormult(T)
    quote
        @eval begin
            (*)(s::$T, x::Vector{Float64}) = s.data .* x
            (*)(x::Vector{Float64}, s::$T) = s.data .* x

            (*)(s::$T, x::Float64) = s.data .* x
            (*)(x::Float64, s::$T) = s.data .* x

            (*)(s::$T, x::Int) = s.data .* Float64(x)
            (*)(x::Int, s::$T) = s.data .* Float64(x)

            (*)(s::$T, x::$T) = s.data .* x.data
        end
    end
end

@vectormult

I have an aversion to copy and paste and I have lots of structs that contain a single vector and a bunch of properties. These structs are just dying to be multiplied by things. The macro above was a easy to write. It is not going to win any prizes for generic coding, but give me a break. This was my first macro and it works for what I needed it for. Here is an example.

In [43]:
import Base: *

struct Amplitude
    data::Vector{Float64}
    alpha::Float64
    beta::Float64
end

@vectormult Amplitude

a = Amplitude([3.2, 1.3], 0.5, 0.33)
2a

2-element Vector{Float64}:
 6.4
 2.6

  

> **Meta lesson 1**. There is no harm in seeking a stable foundation from which to dive into the abyss.

  
  

That first macro is not exactly ground breaking, but it shows that macros can be easy. What makes this one easy is:

1) It is basic template substitution. Julia substitutes T with Amplitude at compile time and produces new code that points to the Amplitude struct.
2) The act of substituation affect what this macro brings back into the main namespace. It always brings back (*).
3) T is a scalar.
4) The input to the macro (T) is not transformed by the macro. It is used as is: $T.

For any macro that conforms to the above criteria, you have nothing to fear. Go forth and mutiply, divide or do anything to avoid the dreaded copy and paste.

  
  
> **Meta lesson 2**. "You know nothing John Snow" - George R. R. Martin. 
  
  

When you consider the corollary of those 4 criteria, it is safe to assume the abyss is deep.


### Julia Expressions

There is a lot of material covering Julia expressions and the abstract syntax tree. I am not going to cover it again. The material is good, but I didn't find an obvious way to tie to the practicalities of macro building.  When attempting a macro that violoates any of the four easy criteria, you need to be aware of what is happening to whatever you pass into your macro. 

 
 
> **Meta lesson 3**. Dive playfully into the abyss without seeking immediate reward.
   
  

#### Get used to taking a peak at expressions

One of my early fears with macros was *print()*. There are examples that show compile time *print()* statements vs runtime *print()* statements. The compile time statements are really useful.

**Pro tip:** Anything you put before the start of the quote block is regular Julia code (free from any macro weirdness) and it only runs at compile time. 

In [44]:
macro vectormult(T)
    @show T
    quote
        @eval begin
            (*)(s::$T, x::Vector{Float64}) = s.data .* x
            (*)(x::Vector{Float64}, s::$T) = s.data .* x

            (*)(s::$T, x::Float64) = s.data .* x
            (*)(x::Float64, s::$T) = s.data .* x

            (*)(s::$T, x::Int) = s.data .* Float64(x)
            (*)(x::Int, s::$T) = s.data .* Float64(x)

            (*)(s::$T, x::$T) = s.data .* x.data
        end
    end
end

@vectormult Amplitude


T = :Amplitude


* (generic function with 325 methods)

When all you pass to a macros is a scalar value or identifier, it easy to understand how the macro sees it.

In [45]:

macro showinput(y)
	show(y)
end

@showinput Amplitude

:Amplitude

The macro sees Amplitude as the symbol :Amplitude

In [46]:
@showinput Amplitude, println

:((Amplitude, println))

The macro sees the input "Amplitude, println" as an expression representation of a tuple. This is how you can create an expression yourself.

In [47]:
x = :((Amplitude, println))

:((Amplitude, println))

The two inputs to the macro were conveniently converted to a tuple and this tuple was converted into an expression. It is no longer a tuple, so all of its tupplness was left behind when it was turned into an expression. It is no longer indexable, so it is not iterable either. You have the power of the julia programming language to operate on it, but it is pretty much powerless. Don't say nobody warned you. Macros are hard!

In [48]:
x[1]

MethodError: MethodError: no method matching getindex(::Expr, ::Int64)

 
> **Meta lesson 4**. Accept the gift of feedback. No matter how cryptic or harsh.

I wish Julia's errors were more like Elm's. They are useful, but to need to put in the time and thought to understand how they are useful. When you understand what they are telling you, they are in fact blindingly obvious.

Isn't it reasonably obvious to interpret the error above as, "You idiot developer, how could you be so dumb as to try to index into an expression. Use your noggin and the docs. You will find the correct way to find the pieces of an expression! ". Instead they give you red herrings about Methods, Ints and getindexes.
  

Note: It is useful to know how to roundtrip things and convert an expression back to Julia objects. You use eval(). Horray! We have an indexable tuple again.

In [49]:
eval(x)[1]

Amplitude

#### Making sense of input expressions

Unless you pass one or more scalars to a macro that you use verbatim without transformation, you will need learn how to decompose expressions into pieces and work with those pieces. This next section gives you some practical ways to get to grips with expressions.

In [50]:
dump(x)

Expr
  head: Symbol tuple
  args: Array{Any}((2,))
    1: Symbol Amplitude
    2: Symbol println


By dumping the expression we get a better view of what is inside an expression. All expressions look the same. They contains a head that explains what type of expression it is and args that explain what the expression operates on. The data structure is a tree. Let's make an expanded one and examine the tree.

In [51]:
exprtree = :(Amplitude, x + 1)
dump(exprtree)

Expr
  head: Symbol tuple
  args: Array{Any}((2,))
    1: Symbol Amplitude
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Symbol x
        3: Int64 1


When you work with the contents of expressions in your macro code, do don't have the convenience of "dump". You can traverse the tree starting with the args of the root.

In [52]:
exprtree.args

2-element Vector{Any}:
 :Amplitude
 :(x + 1)

Since args is a vector we can index into it.

In [53]:
exprtree.args[2].head

:call

You may have expected to see a *+* as the head, but *+* is not an expression type. *+* is a function. All function calls live inside an expression type of *:call*. The args are the function name *+* and the literals that are being added.

In [54]:
exprtree.args[2].args

3-element Vector{Any}:
  :+
  :x
 1

If *x+1* were indeed an example of an input that your needed for a macro and you need to isolate each piece as variable like x, a operation like + and a constant like 1, you could locate these elements by position in the expression.

In [55]:
variable = exprtree.args[2].args[2]
operation = exprtree.args[2].args[1]
constant = exprtree.args[2].args[3]
(variable, operation, constant)

(:x, :+, 1)

This clearly works, and you may call me picky, but I not too keen on using, debugging or maintaining code that looks like this. I wondered whether there would be a way to parse expressions in a way that makes it easier to adapt to variations in the structure of the expression, gives good feedback to the developer when they mess up and supply the wrong input.

### A more methodical way

The example below demonstrates a more repeatable way of hunting down and structuring fragments in expressions.

In [56]:
include("src/MacroExpression.jl")
found = MacroExpression.parseinput([MacroExpression.BinaryFunction{Symbol, Int}], :(x+1))

1-element Vector{Any}:
 Main.MacroExpression.BinaryFunction{Symbol, Int64}(:(x + 1), :+, (Symbol, Int64))

The *parseinput()* function automates the process of looking for patterns in expressions. It can't look for each pattern known to mankind and splat out what it finds - otherwise the macro would have to decide what to do with each pattern known to mankind. Instead *parseinput()* looks for specific patterns that the macro author chooses to accept as inputs to the macro. 

In the example above, I instructed *parseinput* to hunt down *BinaryFunction*s that operate on a Symnol and an integer.  A *BinaryFunction* is defined as a Type. This is what it looks like.

```{Julia}
struct BinaryFunction{T,U}
    expr::Any
    func::Symbol
    argtypes::Tuple{DataType, DataType}
    BinaryFunction{T,U}(e, f::Symbol) where {T, U} = new(e, f, (T, U))
end

function capture(::Type{BinaryFunction{T,U}}, e::Expr) where {T,U}
    if e.head == :call && length(e.args) == 3
        return BinaryFunction{T,U}(e, e.args[1])
    end
    return nothing
end

func(x::BinaryFunction) = x.func

```

There are also a few methods implemented for this type. 

The *capture()* method contains the pattern matching code that finds expressions that look like *BinaryFunctions*. I defined this pattern as something that makes a *:call* with two arguments.

There is also a *func()* method that extracts the function symbol from a *BinaryFunction* object.

### Variations in macro signatures

Notice how *parseinput()* returns a vector. That is because there is another problem that *parseinput()* helps with. Unlike my introductory simple macro that had one scalar input, macros can have much more complex signatures.

#### Use case: Macro has many scalar inputs

The macro below makes two separate calls - one for each input. It doesn't do anything useful. It simply transforms the inputs and returns them as a tuple.

In [57]:
macro twoinputs(a,b)
    input1 = MacroExpression.parseinput([MacroExpression.BinaryFunction{Symbol, Int}], a)
    input2 = MacroExpression.parseinput([MacroExpression.UnaryFunction{Int}], b)
    return quote
        (
            $(MacroExpression.func(input1[1])),
            $(MacroExpression.func(input2[1]))
            )
    end
end
x1 = 23
@twoinputs x1+3 sqrt(9)

(+, sqrt)

The macro above parses the input a for a BinaryFunction and then parses input b for a UnaryFunction. If your macro expects specific types in a specific order, this is the way to do it. You could also loosen up your requirements.

### Use case: Parse inputs by type without considering ordering

In [58]:
macro twoanyorder(a,b)
    types = [MacroExpression.BinaryFunction{Symbol, Int}, MacroExpression.UnaryFunction{Int}]
    input1 = MacroExpression.parseinput(types, a)
    input2 = MacroExpression.parseinput(types, b)
    return quote
        (
            $(MacroExpression.func(input1[1])),
            $(MacroExpression.func(input2[1]))
            )
    end
end
x1 = 23
@twoanyorder sqrt(9) x1+3

(sqrt, +)

The above variant looks for both types in both inputs. Next we will use a vararg input.

### Use case: variable number of inputs

In [59]:
macro varinputs(a...)
    types = [MacroExpression.BinaryFunction{Symbol, Int}, MacroExpression.UnaryFunction{Int}]
    inputs = MacroExpression.parseinput(types, a...)
    outputs = [MacroExpression.func(x) for x in inputs]
    return quote
        $outputs
    end
end
x1 = 23
@varinputs sqrt(9) x1+3 sqrt(12)

3-element Vector{Symbol}:
 :sqrt
 :+
 :sqrt

### Use case: inputs in a begin end block

It is often more convenient for the user of your macro to place multiple inputs in a block. A block needs special parsing. *parseinput()* will look for blocks and unravel them.

In [60]:
@varinputs begin
    sqrt(9)
    x1+3
    sqrt(12)
end

3-element Vector{Symbol}:
 :sqrt
 :+
 :sqrt

Of course the macros defined above don't do anything useful. They only explain a range of possibilities for defining and parsing inputs.

> Meta lesson 5: After you fail at doing something useful, learn by succeeding at doing something useless

This is particularly true with macros. If you code a complex macro and it doesn't work, you you will have a tough time figuring out why it failed because it doesn't behave as predictably as regular code.  For the macros that I have worked on, I break them into 3 parts: parsing inputs ; transforming parsed inputs ; outputing something based on those transfomed parts.

After dividing you macro up this way it is easy to work on each part in isolation: starting with the inputs and outputs parts separately. If you can't capture the inputs you were expecting to capture or can't process the outputs in the way you wanted, there is no sense in wasting time on the transformation.

There is another benefit to breaking up your thinking and your code in this way. It is easier to fall back on what you already know. When you understand exactly what expression each step will get an input and what expression it needs to return as output, you can step out of the abyss and use conventional coding approaches: writing functions outside of macros to transform expressions. 

### Use Case: Wrapping code

A good example is BenchmarkTools @benchmark. It takes some code as input, wrapping it with code that loops its execution and times the result.

```{Julia}
julia> @btime sin(x) setup=(x=rand())
  4.361 ns (0 allocations: 0 bytes)
0.49587200950472454
```

Here is an example of WIP towards a macro that follows the basic structure of @btime. It demontrates how to build a structure to wrap something.

In [61]:
macro wrapitwip(e)
    types = [MacroExpression.BinaryFunction{Symbol, Int}, MacroExpression.UnaryFunction{Int}]
    inputs = MacroExpression.parseinput(types, e)
    ## tranformation TBD
    return quote
        counter = 0
        for i in 1:10
            println("wrapped code will run here")
            counter += 1
        end
        counter
    end
end
@wrapitwip sqrt(9)

wrapped code will run here
wrapped code will run here
wrapped code will run here
wrapped code will run here
wrapped code will run here
wrapped code will run here
wrapped code will run here
wrapped code will run here
wrapped code will run here
wrapped code will run here


10

The above WIP example is a bridge to the next set of use cases. These use cases will explore the types of things you can get macros to return.


### Use case: defining new identifiers in your code

This first one is really basic. It initializes a new variable to a value of 1.

In [62]:
macro setone(x)
    inputs = MacroExpression.parseinput([Symbol], x)
    var = MacroExpression.literal(inputs[1]) |> esc
    some_local_var = 23
    return quote
        $var = 1
    end
end
@setone myvar 
myvar

1

After this macro runs myvar is available outside the macro. You can think of this as the most basic of code generation examples. The quote section generated a simple assignment of myvar to the value of 1.

I threw *some_local_var" so you can see how that is different to $var. Currently it is not used. Let's see what happens if I try to output it.

In [63]:
macro setone(x)
    inputs = MacroExpression.parseinput([Symbol], x)
    var = MacroExpression.literal(inputs[1]) |> esc
    somelocalvar = 23
    return quote
        $var = 1
        somelocalvar
    end
end
@setone myvar 
myvar

1

The way that I understand what happened above is that the whole quote section is like a textual template. It is as if the macro pastes the quote section into the code when the macro runs. When it pastes the piece of text "somelocalvar", we get an error because "somelocalvar" is not in the context when that pasted in piece of text runs.

If we add it to context before the macro runs, look what happens:

In [64]:
somelocalvar = 122
@setone myvar

122

The assigment of *somelocalvar" to 23 inside the macro seems to be ignored. Why is this?

There are 3 reasons:
1) It is assigned at compile time: Anything outside the quote section only runs when the macro is compiled. 
2) It is assigned in a different local context: Julia's "macro hygiene" protects you from overwriting variables by creating a new context for the macro compile process. 
3) It is never actually refered to in the quote section: If I wanted the *somelocalvar* that I defined at compile time to influence the macro at runtime I have to interpolate it at compiletime.



In [65]:
macro setonewithreturn(x)
    inputs = MacroExpression.parseinput([Symbol], x)
    var = MacroExpression.literal(inputs[1]) |> esc
    somelocalvar = 23
    return quote
        $var = 1
        $somelocalvar
    end
end
@setonewithreturn myvar2 

23

Now the macro returns 23 because it was essentially hardcoded into the macro at compile time.  *$somelocalvar* becomes 23 after compilation.

In [67]:
somelocalvar

122

*somelocalvar* is still 122. That is because Julia created a new local variable when it compiled the macro.

In [68]:
myvar2

1

As before, *myvar2* is assigned when the macro runs. It is assigned when interpolating *var*. You will notice that when I assigned *var*, I took the precausion of *esc()*'ing it. I did that in case Juila decided that it the symbol *myvar2* that I passed as an expression to the macro needed to be transported into a new context to prevent it clashing with an existing identifier. In this case my parania was uncalled for. This works without the *|> esc*. 

The docs don't attempt to explain exactly when you need to be paranoid and *esc()* things. I didn't try to figure it out. I look out for errors that suggest missing identifiers and *esc()* when needed.

### Generating a variable number of identifiers in quote code

The previous macro kept things simple by avoiding vectors. Let's throw caution to the wind.


In [69]:
macro setmanytoone(x...)
    inputs = MacroExpression.parseinput([Symbol], x...)
    vars = [:($(MacroExpression.literal(x)) = 1) for x in inputs]
    print(vars)
    return quote
        $(vars...)
    end
end
@setmanytoone myvar3 myvar4
myvar3


Expr[:(myvar3 = 1), :(myvar4 = 1)]

UndefVarError: UndefVarError: myvar3 not defined

Of all of the grey hair inducing moments of this endevour, the one line of code that produces *vars* above was the worst.

I will work backwards to explain it.

1) To produce a variable number of lines of code when your macro runs you have to interpolate a vector containing the expressions that you want "pasted in your code".
2) This means you have to figure out what kind of vector will produce the right result when interpolated.

This is were it pays to step away from your macro and do something like this:

In [70]:
dump(:(somevar1 = 2, somevar2 = 7))

Expr
  head: Symbol tuple
  args: Array{Any}((2,))
    1: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol somevar1
        2: Int64 2
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol somevar2
        2: Int64 7


$vars needs to look something like the above.

What kind of transformation will produce this from an array containing the two symbols?

In [71]:
symbols = [:somevar1, :somevar2]
[x=1 for x in symbols]

2-element Vector{Int64}:
 1
 1

That didn't work. "x=1" produces [1,1]

In [66]:
[:(x = 1) for x in symbols]

2-element Vector{Expr}:
 :(x = 1)
 :(x = 1)

At least we have 2 expressions, but remember that long and boring story about *somelocalvar*? It only existed at compile time until we interpolated it. We don't want poor *x* to suffer the same fate. We want the value of x at compile time inserted into the macro so we will need to interpolate it into the macro. 

In [72]:
[:($x = 1) for x in symbols]

2-element Vector{Expr}:
 :(somevar1 = 1)
 :(somevar2 = 1)